In [ ]:
!pip install requests pandas numpy

In [15]:
import requests
from bs4 import BeautifulSoup

def get_imdb_reviews(movie_id):
    url = f"https://www.imdb.com/title/{movie_id}/reviews"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 403:
        print("Access denied. Try using a different User-Agent or a proxy.")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    reviews = []
    for review in soup.find_all('div', class_=''):
        reviews.append(review.get_text())

    return reviews

movie_id = 'tt0111161'  # ID filmu "The Shawshank Redemption"
reviews = get_imdb_reviews(movie_id)
for i, review in enumerate(reviews, 1):
    print(f"Review {i}: {review}\n")

Review 1:  




Review 2: MenuMoviesRelease CalendarTop 250 MoviesMost Popular MoviesBrowse Movies by GenreTop Box OfficeShowtimes & TicketsMovie NewsIndia Movie SpotlightTV ShowsWhat's on TV & StreamingTop 250 TV ShowsMost Popular TV ShowsBrowse TV Shows by GenreTV NewsWatchWhat to WatchLatest TrailersIMDb OriginalsIMDb PicksIMDb SpotlightIMDb PodcastsAwards & EventsOscarsSXSW Film FestivalWomen's History MonthSTARmeter AwardsAwards CentralFestival CentralAll EventsCelebsBorn TodayMost Popular CelebsCelebrity NewsCommunityHelp CenterContributor ZonePollsFor Industry ProfessionalsLanguageEnglish (United States)LanguageFully supportedEnglish (United States)Partially supportedFrançais (Canada)Français (France)Deutsch (Deutschland)हिंदी (भारत)Italiano (Italia)Português (Brasil)Español (España)Español (México)AllAllWatchlistSign InENFully supportedEnglish (United States)Partially supportedFrançais (Canada)Français (France)Deutsch (Deutschland)हिंदी (भारत)Italiano (Italia)Português (Brasil)